In [0]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/WashingtonPost.v2/data/parquet"

ContentsType-animated_video.parquet  ContentsType-kicker.parquet
ContentsType-ar-wikitude.parquet     ContentsType-list.parquet
ContentsType-author_info.parquet     ContentsType-pull_quote.parquet
ContentsType-byline.parquet	     ContentsType-sanitized_html.parquet
ContentsType-date.parquet	     ContentsType-title.parquet
ContentsType-deck.parquet	     ContentsType-top_deck.parquet
ContentsType-gallery.parquet	     ContentsType-tweet.parquet
ContentsType-image.parquet	     ContentsType-video.parquet
ContentsType-inline_story.parquet    documents.parquet
ContentsType-instagram.parquet


# Load Data

In [0]:
filepath = "/content/drive/My Drive/WashingtonPost.v2/data/parquet/ContentsType-sanitized_html.parquet"
doc_df = pd.read_parquet(filepath)
print(doc_df.shape)

#Group by Document ID

In [0]:
# Append all text from a document together
doc_df_grouped = doc_df.groupby('document_id')['content'].apply(lambda x: " ".join(x))

In [0]:
filepath_data_grouped = "/content/drive/My Drive/WashingtonPost.v2/data/doc_grouped.csv"
doc_df_grouped.to_csv(filepath_data_grouped)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [0]:
doc_df_grouped.head()

document_id
0000107b869682826003b04a40e63948        If Senate Democrats needed a miracle not to lo...
00002df2-867b-11e1-abb4-839961045500    <span class="dateline">CHEYENNE, Wyo. —</span>...
000045ed8eabe76d1049d21dfc08abbd        <strong>ON THE DOCKET: </strong>A three-judge ...
0000559e4f5786355e61755bd3bac097        The last few years have not been kind to the V...
000058a975a5d3bceec214776c48d7a2        Kris Bryant, the second overall pick two years...
Name: content, dtype: object

# Load Grouped Documents

In [0]:
filepath = "/content/drive/My Drive/WashingtonPost.v2/data/doc_grouped.csv"
doc_df = pd.read_csv(filepath, header=None)

In [0]:
print(doc_df.shape)
print(len(doc_df))
print(doc_df.head())

(592341, 2)
592341
                                      0                                                  1
0      0000107b869682826003b04a40e63948  If Senate Democrats needed a miracle not to lo...
1  00002df2-867b-11e1-abb4-839961045500  <span class="dateline">CHEYENNE, Wyo. —</span>...
2      000045ed8eabe76d1049d21dfc08abbd  <strong>ON THE DOCKET: </strong>A three-judge ...
3      0000559e4f5786355e61755bd3bac097  The last few years have not been kind to the V...
4      000058a975a5d3bceec214776c48d7a2  Kris Bryant, the second overall pick two years...


In [0]:
!pip install rank_bm25

  Created wheel for rank-bm25: filename=rank_bm25-0.2-cp36-none-any.whl size=4162 sha256=75017f9c968d75b84b4474588b3f6bc0deb9c696ae28642b86e3d2120d848df0
  Stored in directory: /root/.cache/pip/wheels/6f/0c/1f/78945dd6a5478bbcdb50d73ac96ae5af2ffcdfcd374fd9b1bf
Successfully built rank-bm25


# Preprocess Data


In [0]:
## Tokenize:
import numpy as np
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')
from bs4 import BeautifulSoup

In [0]:
# Tokenize sentence. Remove punctuation marks.
# Converts all words to lowercase
def tokenize(sentence):
    sentence = sentence.replace('’','')
    words = nltk.word_tokenize(sentence)
    # Remove single character punctuation words
    words_cleaned = [ word.lower() for word in words if word.isalnum() or not len(word) == 1 ]
    return words_cleaned

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
filepath_tokenized = "/content/drive/My Drive/WashingtonPost.v2/data/tokenized_data.csv"

index = 0
datasize = len(doc_df)

while index < datasize:
    new_index = index + 2000
    print("Tokenized slice ", new_index)
    print(100*index/datasize, "%")

    # select slice from data
    df_slice = doc_df.iloc[index:new_index]
    index = new_index
    
    id_series = df_slice.iloc[:,0]
    text_series = df_slice.iloc[:,1]

    # Remove html tags from documents
    s_parsed = text_series.map(lambda x: BeautifulSoup(x).get_text())
    print("Parsed")
    # Tokenize the corpus
    tokenized_corpus_slice = s_parsed.map(lambda x: tokenize(x))
    print("Tokenized")
    # Append slice to csv file
    slice_df = pd.DataFrame(dict(s1 = id_series, s2 = tokenized_corpus_slice))
    slice_df.to_csv(filepath_tokenized, header = False, mode="a")
    print("Saved slice")